In [1]:
# -*- coding: utf-8 -*
import os
import re
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import keras
import warnings
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.optimizers import RMSprop
from keras.utils import to_categorical
from keras.backend import tensorflow_backend
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import scipy.linalg as LA
import cv2
import tensorflow as tf

/home/tsunei/anaconda3/envs/tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
config = tf.ConfigProto(gpu_options = tf.GPUOptions(allow_growth = True))
session = tf.Session(config = config)
tensorflow_backend.set_session(session)

In [3]:
def list_pictures(directory, ext='jpg|jpeg|bmp|png|ppm'):
    return [os.path.join(root, f)
            for root, _, files in os.walk(directory) for f in files
            if re.match(r'([\w]+\.(?:' + ext + '))', f.lower())]

In [4]:
X_eva = []
Y_eva = []

#テストデータの所得
print("人工物画像のテストデータを取得してます...")
for filepath in list_pictures('../data/CNN_data/test/jinkou/'):
    warnings.filterwarnings('ignore')
    img_eva = img_to_array(load_img(filepath, target_size=(128, 128)))        
    X_eva.append(img_eva)
    Y_eva.append(0) # 教師データ（正解）

print("人工物画像のテストデータを取得しました")

    
print("自然物画像のテストデータを取得してます...")
for filepath in list_pictures('../data/CNN_data/test/sizen/'):
    img_eva = img_to_array(load_img(filepath, target_size=(128, 128)))
    X_eva.append(img_eva)
    Y_eva.append(1) # 教師データ（正解）


# NumPy配列に変換
X_eva = np.asarray(X_eva)
Y_eva = np.asarray(Y_eva)


# float32型に変換
X_eva = X_eva.astype('float32')

# 正規化(0～1)
X_eva = X_eva / 255.0


人工物画像のテストデータを取得してます...
人工物画像のテストデータを取得しました
自然物画像のテストデータを取得してます...


In [5]:
print("テストデータをzca白色化しています...")
#テストデータにzca白色化
datagen = ImageDataGenerator(zca_whitening=True)
g = datagen.flow(X_eva, Y_eva, X_eva.shape[0], shuffle=False)
x_test, y_test = g.next()
print("テストデータをzca白色化しました")

テストデータをzca白色化しています...
テストデータをzca白色化しました


In [9]:
batch_size = 32 # バッチサイズ(データサイズ)
num_classes = 2 # 分類クラス数(今回は0～9の手書き文字なので10)
epochs = 10 # エポック数(学習の繰り返し回数)
ave = 0
fold_num = 3
seed = 5
np.random.seed(seed)
count = 0

kfold = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
jinkou_score = []
sizen_score = []
cvscores = []

for train, test in kfold.split(x_test, y_test):
    count = count + 1
    print(count,"/",fold_num)
    
    # create model
    model = Sequential()

    # Dense：全結合のニューラルネットワークレイヤー
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(128, 128, 3)))
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    #model.add(Dropout(0.2)) # 過学習防止用：入力の20%を0にする（破棄）
    model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu')) # 活性化関数：relu
    model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    #model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    #model.add(Dropout(0.2)) # 過学習防止用：入力の20%を0にする（破棄）
    model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu')) # 活性化関数：relu
    model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu')) # 活性化関数：relu
    model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu')) # 活性化関数：relu
    model.add(layers.BatchNormalization())
    #model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    #model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    #model.add(Dropout(0.2)) # 過学習防止用：入力の20%を0にする（破棄）
    model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu')) # 活性化関数：relu
    model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    #model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    #model.add(Dropout(0.2)) # 過学習防止用：入力の20%を0にする（破棄）
    model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu')) # 活性化関数：relu
    model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    #model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    #model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    #model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    #model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    #model.add(Dropout(0.2)) # 過学習防止用：入力の20%を0にする（破棄）
    model.add(layers.Dense(num_classes, activation='sigmoid')) # 活性化関数：softmax

    #model.summary()

    # コンパイル（多クラス分類問題）
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    # 構築したモデルで学習
    model.fit(x_test[train], np_utils.to_categorical(y_test[train], 2), batch_size=batch_size, epochs=epochs)

    y_pred = model.predict_classes(x_test[test])

    sum_all_j = 0
    sum_all_s = 0
    sum_j = 0
    sum_s = 0

    for i in range(x_test[test].shape[0]):
        #print("%s:%s" % (y_pred[i], Y_eva[test][i]))
        if y_pred[i] == 1:
            if y_pred[i] == y_test[test][i]:           
                sum_j = sum_j + 1  

            sum_all_j = sum_all_j + 1  

        else:
            if y_pred[i] == y_test[test][i]:
                sum_s = sum_s + 1

            sum_all_s = sum_all_s + 1


    print("人工物画像の正解：",sum_j,"/",sum_all_j,"＝",sum_j / sum_all_j * 100,"%")
    print("自然物画像の正解：",sum_s,"/",sum_all_s,"＝",sum_s / sum_all_s * 100,"%")
    
    sum_j =  sum_j / sum_all_j * 100
    sum_s =  sum_s / sum_all_s * 100
    ave = (sum_s + sum_j) / 2 + ave
      
    # Evaluate
    scores = model.evaluate(x_test[test], keras.utils.to_categorical(y_test[test], num_classes), verbose=0)
    #print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    #cvscores.append(scores[1] * 100)
    jinkou_score.append(sum_j)
    sizen_score.append(sum_s)
    cvscores.append((sum_s + sum_j) / 2)
    print(" ")

for i in range(fold_num):
    print((i+1),"/",fold_num)
    print("人工物画像の正解率＝",jinkou_score[i],"%")
    print("自然物画像の正解率＝",sizen_score[i],"%")
    print("全体画像の正解率＝",cvscores[i],"%")

print(" ")
print("平均正解率：%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

1 / 3
Epoch 1/10
1600/1600 [==============================] - 8s 5ms/step - loss: 1.6170 - acc: 0.6109
Epoch 2/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.6005 - acc: 0.6769
Epoch 3/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.6431 - acc: 0.6759
Epoch 4/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.5986 - acc: 0.6778
Epoch 5/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.6025 - acc: 0.6897
Epoch 6/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.5959 - acc: 0.7137
Epoch 7/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.5059 - acc: 0.7669
Epoch 8/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.4530 - acc: 0.7919
Epoch 9/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.4524 - acc: 0.7881
Epoch 10/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.5321 - acc: 0.7869
人工物画像の正解： 340 / 452 ＝ 7